In [0]:
%sql
CREATE VIEW `hls_sql_workshop`.`gold`.`claims_metric_view`
(
  `Claim ID` COMMENT "Unique identifier for each claim",
  `Claim Date` COMMENT "Date of the claim",
  `Claim Month` COMMENT "Month of the claim date",
  `Claim Year` COMMENT "Year of the claim date",
  `Beneficiary ID` COMMENT "Unique identifier for the beneficiary",
  `Beneficiary Gender` COMMENT "Gender of the beneficiary",
  `Diagnosis Code` COMMENT "Primary diagnosis code for the claim",
  `Diagnosis Description` COMMENT "Description of the diagnosis code",
  `Provider ID` COMMENT "Unique identifier for the provider",
  `Provider Entity Type` COMMENT "Provider's entity type",
  `Total Claim Amount` COMMENT "Sum of claim amounts",
  `Average Claim Amount` COMMENT "Average claim amount per claim",
  `Distinct Beneficiaries` COMMENT "Number of unique beneficiaries",
  `Claims Count` COMMENT "Total number of claims",
  `Trailing 30 Day Claim Amount` COMMENT "Sum of claim amounts in trailing 30 days"
)
WITH METRICS
LANGUAGE YAML
AS $$
version: 0.1

source: hls_sql_workshop.gold.fact_patient_claims

joins:
  - name: date
    source: hls_sql_workshop.gold.dim_date
    on: claim_start_date = date.date

  - name: beneficiary
    source: hls_sql_workshop.gold.dim_beneficiary
    on: source.beneficiary_key = beneficiary.beneficiary_key

  - name: diagnosis
    source: hls_sql_workshop.gold.dim_diagnosis
    on: source.diagnosis_key_1 = diagnosis.diagnosis_key

  - name: provider
    source: hls_sql_workshop.gold.dim_provider
    on: source.attending_physician_provider_key = provider.provider_key

dimensions:
  - name: Claim ID
    expr: claim_id

  - name: Claim Date
    expr: claim_start_date

  - name: Claim Month
    expr: date_trunc('month', claim_start_date)

  - name: Claim Year
    expr: date_trunc('year', claim_start_date)

  - name: Beneficiary ID
    expr: beneficiary_key

  - name: Beneficiary Gender
    expr: beneficiary.gender

  - name: Diagnosis Code
    expr: diagnosis.diagnosis_code

  - name: Diagnosis Description
    expr: diagnosis.diagnosis_short_description

  - name: Provider ID
    expr: provider.provider_key

  - name: Provider Entity Type
    expr: provider.entity_type

measures:
  - name: Total Claim Amount
    expr: SUM(claim_payment_amount)

  - name: Average Claim Amount
    expr: AVG(claim_payment_amount)

  - name: Distinct Beneficiaries
    expr: COUNT(DISTINCT beneficiary_key)

  - name: Claims Count
    expr: COUNT(claim_id)

  - name: Trailing 30 Day Claim Amount
    expr: SUM(claim_payment_amount)
    window:
      - order: Claim Date
        range: trailing 30 day
        semiadditive: last
$$